<h1> Structured data prediction using Cloud AI Platform </h1>

This notebook illustrates:
<ol>
<li> Create a BigQuery Dataset and Google Cloud Storage Bucket 
<li> Export from BigQuery to CSVs in GCS
<li> Training on Cloud AI Platform
<li> Deploy trained model
</ol>

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
!pip install --user google-cloud-bigquery==3.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.1/215.1 kB 4.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.22.3 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.22.3 which is incompatible.
tensorflow-transform 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.22.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.22.3 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.22.3 which is incompatible.
tensorboardx 2.6 requires protobuf<4,>=3.8.0, but yo

**Note**: Restart your kernel to use updated packages.

**Kindly ignore the deprecation warnings and incompatibility errors related to google-cloud-storage**.

## Set up environment variables and load necessary libraries

Set environment variables so that we can use them throughout the entire notebook. We will be using our project name for our bucket, so you only need to change your project and region.

In [3]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-01-4a84667811dc' # Replace with the your bucket name
PROJECT = 'qwiklabs-gcp-01-4a84667811dc' # Replace with your project-id
REGION = 'us-central1'

In [4]:
import os

from google.cloud import bigquery
import time

In [5]:
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "2.3"
os.environ["PYTHONVERSION"] = "3.7"

In [6]:
%%bash
export PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-01-4a84667811dc


## The source dataset

Our dataset is hosted in [BigQuery](https://cloud.google.com/bigquery/). The CDC's Natality data has details on US births from 1969 to 2008 and is a publically available dataset, meaning anyone with a GCP account has access. Click [here](https://console.cloud.google.com/bigquery?project=bigquery-public-data&p=publicdata&d=samples&t=natality&page=table) to access the dataset.

The natality dataset is relatively large at almost 138 million rows and 31 columns, but simple to understand. `weight_pounds` is the target, the continuous value we’ll train a model to predict.

## Create a BigQuery Dataset and Google Cloud Storage Bucket 

A BigQuery dataset is a container for tables, views, and models built with BigQuery ML. Let's create one called __babyweight__. We'll do the same for a GCS bucket for our project too.

In [7]:
%%bash

# Create a BigQuery dataset for babyweight if it doesn't exist
datasetexists=$(bq ls -d | grep -w babyweight)

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset already exists, let's not recreate it."

else
    echo "Creating BigQuery dataset titled: babyweight"
    
    bq --location=US mk --dataset \
        --description "Babyweight" \
        $PROJECT:babyweight
    echo "Here are your current datasets:"
    bq ls
fi
    
## Create GCS bucket if it doesn't exist already...
exists=$(gsutil ls -d | grep -w gs://${BUCKET}/)

if [ -n "$exists" ]; then
    echo -e "Bucket exists, let's not recreate it."
    
else
    echo "Creating a new GCS bucket."
    gsutil mb -l ${REGION} gs://${BUCKET}
    echo "Here are your current buckets:"
    gsutil ls
fi

Creating BigQuery dataset titled: babyweight
Dataset 'qwiklabs-gcp-01-4a84667811dc:babyweight' successfully created.
Here are your current datasets:
  datasetId   
 ------------ 
  babyweight  
Bucket exists, let's not recreate it.


## Create the training and evaluation data tables

Since there is already a publicly available dataset, we can simply create the training and evaluation data tables using this raw input data. First we are going to create a subset of the data limiting our columns to `weight_pounds`, `is_male`, `mother_age`, `plurality`, and `gestation_weeks` as well as some simple filtering and a column to hash on for repeatable splitting.

* Note:  The dataset in the create table code below is the one created previously, e.g. "babyweight".

### Preprocess and filter dataset

We have some preprocessing and filtering we would like to do to get our data in the right format for training.

Preprocessing:
* Cast `is_male` from `BOOL` to `STRING`
* Cast `plurality` from `INTEGER` to `STRING` where `[1, 2, 3, 4, 5]` becomes `["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)"]`
* Add `hashcolumn` hashing on `year` and `month`

Filtering:
* Only want data for years later than `2000`
* Only want baby weights greater than `0`
* Only want mothers whose age is greater than `0`
* Only want plurality to be greater than `0`
* Only want the number of weeks of gestation to be greater than `0`

In [8]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data AS
SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    mother_age,
    CASE
        WHEN plurality = 1 THEN "Single(1)"
        WHEN plurality = 2 THEN "Twins(2)"
        WHEN plurality = 3 THEN "Triplets(3)"
        WHEN plurality = 4 THEN "Quadruplets(4)"
        WHEN plurality = 5 THEN "Quintuplets(5)"
    END AS plurality,
    gestation_weeks,
    FARM_FINGERPRINT(
        CONCAT(
            CAST(year AS STRING),
            CAST(month AS STRING)
        )
    ) AS hashmonth
FROM
    publicdata.samples.natality
WHERE
    year > 2000
    AND weight_pounds > 0
    AND mother_age > 0
    AND plurality > 0
    AND gestation_weeks > 0

Query is running:   0%|          |

""


### Augment dataset to simulate missing data

Now we want to augment our dataset with our simulated babyweight data by setting all gender information to `Unknown` and setting plurality of all non-single births to `Multiple(2+)`.

In [9]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_augmented_data AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    hashmonth
FROM
    babyweight.babyweight_data
UNION ALL
SELECT
    weight_pounds,
    "Unknown" AS is_male,
    mother_age,
    CASE
        WHEN plurality = "Single(1)" THEN plurality
        ELSE "Multiple(2+)"
    END AS plurality,
    gestation_weeks,
    hashmonth
FROM
    babyweight.babyweight_data

Query is running:   0%|          |

""


### Split augmented dataset into train and eval sets

Using `hashmonth`, apply a module to get approximately a 75/25 train-eval split.

#### Split augmented dataset into train dataset

In [10]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data_train AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    babyweight.babyweight_augmented_data
WHERE
    ABS(MOD(hashmonth, 4)) < 3

Query is running:   0%|          |

""


#### Split augmented dataset into eval dataset

In [11]:
%%bigquery
CREATE OR REPLACE TABLE
    babyweight.babyweight_data_eval AS
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    babyweight.babyweight_augmented_data
WHERE
    ABS(MOD(hashmonth, 4)) = 3

Query is running:   0%|          |

""


## Verify table creation

Verify that you created the dataset and training data table.

In [12]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_train
LIMIT 0

Query is running:   0%|          |

Downloading: |          |

,weight_pounds,is_male,mother_age,plurality,gestation_weeks


In [13]:
%%bigquery
-- LIMIT 0 is a free query; this allows us to check that the table exists.
SELECT * FROM babyweight.babyweight_data_eval
LIMIT 0

Query is running:   0%|          |

Downloading: |          |

,weight_pounds,is_male,mother_age,plurality,gestation_weeks


## Export from BigQuery to CSVs in GCS

Use BigQuery Python API to export our train and eval tables to Google Cloud Storage in the CSV format to be used later for TensorFlow/Keras training. We'll want to use the dataset we've been using above as well as repeat the process for both training and evaluation data.

In [14]:
# Construct a BigQuery client object.
client = bigquery.Client()

dataset_name = "babyweight"

# Create dataset reference object
dataset_ref = client.dataset(
    dataset_id=dataset_name, project=client.project)

# Export both train and eval tables
for step in ["train", "eval"]:
    destination_uri = os.path.join(
        "gs://", BUCKET, dataset_name, "data", "{}*.csv".format(step))
    table_name = "babyweight_data_{}".format(step)
    table_ref = dataset_ref.table(table_name)
    extract_job = client.extract_table(
        table_ref,
        destination_uri,
        # Location must match that of the source table.
        location="US",
    )  # API request
    extract_job.result()  # Waits for job to complete.

    print("Exported {}:{}.{} to {}".format(
        client.project, dataset_name, table_name, destination_uri))

Exported qwiklabs-gcp-01-4a84667811dc:babyweight.babyweight_data_train to gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/train*.csv
Exported qwiklabs-gcp-01-4a84667811dc:babyweight.babyweight_data_eval to gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval*.csv


## Verify CSV creation

Verify that we correctly created the CSV files in our bucket.

In [15]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/data/*.csv

gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000000.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000001.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000002.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000003.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000004.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000005.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000006.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000007.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000008.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000009.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000010.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/train000000000000.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/train000000000001.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/train000000000002.csv
gs:

## Check data exists

Verify that you previously created CSV files we'll be using for training and evaluation.

In [16]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/data/*000000000000.csv

gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/eval000000000000.csv
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/data/train000000000000.csv


## Custom Training on Vertex AI

Now that we see everything is working locally, it's time to train on the cloud! 

To submit to the Cloud we use [`gcloud ai custom-jobs create`](https://cloud.google.com/sdk/gcloud/reference/ai/custom-jobs/create) and simply specify some additional parameters.

Below the `-- \` we add in the arguments for our `task.py` file.

In [17]:
%%bash

gcloud artifacts repositories create babyweight \
    --repository-format=Docker \
    --location=${REGION} \
    --description="repository for babyweight model images" \

Create request issued for: [babyweight]
Waiting for operation [projects/qwiklabs-gcp-01-4a84667811dc/locations/us-central1/operations/2d968299-db5d-4770-8918-f05a804f39ca] to complete...
...............................................................done.
Created repository [babyweight].


In [18]:
IMAGE_URI = f"{REGION}-docker.pkg.dev/{PROJECT}/babyweight/babyweight-image"
os.environ["IMAGE_URI"] = IMAGE_URI

In [19]:
IMAGE_URI

'us-central1-docker.pkg.dev/qwiklabs-gcp-01-4a84667811dc/babyweight/babyweight-image'

In [20]:
%%bash

gcloud artifacts repositories describe babyweight \
    --location=us-central1

createTime: '2023-04-20T14:36:26.775614Z'
description: repository for babyweight model images
dockerConfig: {}
format: DOCKER
mode: STANDARD_REPOSITORY
name: projects/qwiklabs-gcp-01-4a84667811dc/locations/us-central1/repositories/babyweight
updateTime: '2023-04-20T14:36:26.775614Z'


Encryption: Google-managed key
Repository Size: 0.000MB


In [21]:
%%bash

gcloud auth configure-docker ${REGION}-docker.pkg.dev --project=${PROJECT}


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
After update, the following will be written to your Docker config file located 
at [/home/jupyter/.docker/config.json]:
 {
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}

Do you want to continue (Y/n)?  
Docker configuration file updated.


In [22]:
%%bash

OUTDIR=gs://${BUCKET}/babyweight/trained_model
JOBID=babyweight_$(date -u +%y%m%d_%H%M%S)
OUTPUT_IMAGE_URI=${IMAGE_URI}

gcloud ai custom-jobs create \
    --display-name=${JOBID} \
    --region=${REGION} \
    --worker-pool-spec=replica-count=1,machine-type='n1-standard-8',executor-image-uri=us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-11:latest,local-package-path=$(pwd)/babyweight/trainer,script=task.py,output-image-uri=${OUTPUT_IMAGE_URI} \
    --args=--train_data_path=gs://${BUCKET}/babyweight/data/train*.csv,--eval_data_path=gs://${BUCKET}/babyweight/data/eval*.csv,--output_dir=${OUTDIR},--num_epochs=10,--train_examples=10000,--eval_steps=100,--batch_size=32,--nembeds=8
echo


Sending build context to Docker daemon  17.45kB
Step 1/8 : FROM us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-11:latest
latest: Pulling from vertex-ai/training/tf-cpu.2-11
47c764472391: Pulling fs layer
3b4c9936d49d: Pulling fs layer
93d0bde73b80: Pulling fs layer
4f4fb700ef54: Pulling fs layer
ea0acd06b583: Pulling fs layer
4f4fb700ef54: Waiting
aa8facca7254: Pulling fs layer
ea0acd06b583: Waiting
aa8facca7254: Waiting
7b3b020a3528: Pulling fs layer
bd3fc45f53bc: Pulling fs layer
f24ea979b0a7: Pulling fs layer
7b3b020a3528: Waiting
bd3fc45f53bc: Waiting
ec1295410f17: Pulling fs layer
7dde9985aedd: Pulling fs layer
f24ea979b0a7: Waiting
ec1295410f17: Waiting
e2a128ccb651: Pulling fs layer
bbb061b8a3ff: Pulling fs layer
4c0368833e86: Pulling fs layer
7dde9985aedd: Waiting
e2a128ccb651: Waiting
bbb061b8a3ff: Waiting
5174bf6219f1: Pulling fs layer
f9f17923d08d: Pulling fs layer
4c0368833e86: Waiting
5174bf6219f1: Waiting
0c3428ecf4ca: Pulling fs layer
c080dc6d97d3: Pulling fs layer
261fce

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
/usr/lib/google-cloud-sdk/platform/bundledpythonunix/lib/python3.9/subprocess.py:935: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/usr/lib/google-cloud-sdk/platform/bundledpythonunix/lib/python3.9/subprocess.py:941: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)

A custom container image is built locally.

/usr/lib/google-cloud-sdk/platform/bundledpythonunix/lib/python3.9/subprocess.py:935: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdin = io.open(p2cwrite, 'wb', bufsize)
/usr/lib/google-cloud-sdk/platform/bundledpythonunix/lib/python3.9/subprocess.py:941: RuntimeWarning: line buffering (buffering=1) isn't supported 

The training job should complete within 15 to 20 minutes. To see the progress, in your Cloud Console, click on **Navigation menu > Vertex AI > Training > Custom Jobs**. Note that it may take 10 minutes for the job to show up in the list of active jobs. 

When the job has a status of *Finished*, the trained model files are ready for you to use in the next section.

If you already have a trained model, you do not need to wait for this training job to finish before moving forward in the notebook.

## Check our trained model files

Let's check the directory structure of our outputs of our trained model in folder we exported. We'll want to deploy the saved_model.pb within the timestamped directory as well as the variable values in the variables folder. Therefore, we need the path of the timestamped directory so that everything within it can be found by Cloud AI Platform's model deployment service.

In [24]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model

gs://qwiklabs-gcp-01-4a84667811dc/babyweight/trained_model/
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/trained_model/20230420150237/
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/trained_model/checkpoints/


In [25]:
%%bash
MODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/babyweight/trained_model/2* \
                 | tail -1)
gsutil ls ${MODEL_LOCATION}

gs://qwiklabs-gcp-01-4a84667811dc/babyweight/trained_model/20230420150237/
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/trained_model/20230420150237/fingerprint.pb
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/trained_model/20230420150237/saved_model.pb
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/trained_model/20230420150237/assets/
gs://qwiklabs-gcp-01-4a84667811dc/babyweight/trained_model/20230420150237/variables/


## Deploy trained model

Deploying the trained model to act as a REST web service is a simple gcloud call.

In [26]:
ENDPOINT_NAME="babyweight_endpoint"
MODEL_NAME="babyweight"
MODEL_ID="babyweight_"+time.strftime("%Y%m%d-%H%M%S")
ENDPOINT_ID="babyweight"+time.strftime("%Y%m%d-%H%M%S")+"a"
os.environ["ENDPOINT_NAME"] = ENDPOINT_NAME
os.environ["MODEL_NAME"] = MODEL_NAME
os.environ["MODEL_ID"] = MODEL_ID
os.environ["ENDPOINT_ID"] = ENDPOINT_ID

In [27]:
%%bash
gcloud ai endpoints create --display-name=${ENDPOINT_NAME} --region=${REGION} --endpoint-id=${ENDPOINT_ID}

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [400653034910121984]...
..........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

The endpoint should be created within 10 to 15 minutes.

**Note:** If you are unable to access the generated endpoint link, please ignore it. To see the progress, in your Cloud Console, click on **Navigation menu > Vertex AI > Endpoints**.

When the endpoint has a status of *Ready*, the endpoint has been created successfully, and you can proceed to the next cells.

In [29]:
%%bash
gcloud ai models upload --container-image-uri=${IMAGE_URI} \
--display-name=${MODEL_NAME} \
--model-id=${MODEL_ID} \
--region=${REGION}


Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [5307324828930277376]...
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................done.


In [ ]:
%%bash

gcloud ai endpoints deploy-model ${ENDPOINT_ID} --project=${PROJECT} \
    --region=${REGION} \
    --model=${MODEL_ID} \
    --display-name=${MODEL_NAME}

The endpoint should be deployed within 10 to 15 minutes.

To see the progress, in your Cloud Console, click on **Navigation menu > Vertex AI > Endpoints**.

When the endpoint has a status of *Active*, the endpoint has been deployed successfully,

Copyright 2023 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License